In [1]:
from bs4 import BeautifulSoup
# import pandas as pd
from datetime import datetime, timedelta
from itertools import combinations
import json
import requests
import math
from typing import Optional, List, Dict

from pydantic import BaseModel
from typing import Mapping
import numpy as np
import polars as pl
import math

from shared.utils import formalize_data
from shared.random_rules import RandomRules


In [2]:
a = formalize_data("09 12 27 33 24 30")
print(a)

('091224273033', (9, 12, 24, 27, 30, 33))


In [3]:
rr = RandomRules(
  TOTAL_NUMBERS = 40,
  CHOOSE = 6,
  MAX_PRIMES = 3,
  N_VALUES=["1", "2", "3"],
  # Rangos de valores de las bolillas
  MIN_B_VALUES = [1, 2, 3, 4, 5, 6],
  MAX_B_VALUES = [35, 36, 37, 38, 39, 40]
)



In [4]:
url = 'https://resultados.latinka.com.pe/i.do?m=historico&t=0&s=42'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

tables = soup.select('table')
rows = tables[1].find_all('tr') 


In [6]:
lst = []
for row in rows:
  
  
  for col_index in [2, 3]:
    cols = [data.text.strip() for data in row.find_all('td')]
    if (cols[col_index] == ''):
      continue
  
    id, bolillas_int = formalize_data(bolillas_str=cols[col_index])
    cols.append(id)
    cols.append(bolillas_int)
    cols.extend([b for b in bolillas_int])
    
    is_rare, reason = rr.is_rare_combination(bolillas_int, id)
    cols.append(is_rare)
    cols.append(reason)
    left_scores = [round(b, 4) for b in rr.calculate_left_scores(bolillas_int)]

    right_scores = [round(b, 4) for b in rr.calculate_right_scores(bolillas_int)]
    lr_scores = [round(left + right,4) for left, right in zip(left_scores, right_scores)]
    row_score = round(sum(lr_scores),4)

    shared_values = list(set(left_scores[0:5]) & set(right_scores[1:6]))
    shared_values = [round(s, 4) for s in shared_values]
    cols.extend([score for score in left_scores])
    cols.extend([score for score in right_scores])
    # cols.extend(lr_scores)
    cols.append(shared_values)
    cols.append(len(shared_values))
    cols.append(row_score)
    cols.append(rr.jumps_map(bolillas_int))
    cols.append(rr.unijump(bolillas_int))
    
    lst.append(cols)

with open('./kbl_cache.json', 'w') as file: 
  json.dump(lst, file)

# current_count = len(lst[0])
# for q in lst:
#   if (len(q) != current_count):
#     print(f'ERROR: {q}')

dfRows = pl.DataFrame(
  lst,
  schema=[
    'fecha', 'sorteo', 'bolillas regulares', 'bolillas chauchamba', 'cc1', 'cc2',
    'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6',
    'is_rare_combination', 'reason',
    "score_left_b1", "score_left_b2", "score_left_b3", "score_left_b4", "score_left_b5", "score_left_b6",
    "score_r8_b1", "score_r8_b2", "score_r8_b3", "score_r8_b4", "score_r8_b5", "score_r8_b6",
    # "score_lr_b1", "score_lr_b2", "score_lr_b3", "score_lr_b4", "score_lr_b5", "score_lr_b6",
    "shared_values", "shared_values_count",
    "score_row",
    "jumps_map", "unijump"
  ],
  schema_overrides={
    'combo': pl.Array(pl.UInt8, 6),
    'b1': pl.UInt8,
    'b2': pl.UInt8,
    'b3': pl.UInt8,
    'b4': pl.UInt8,
    'b5': pl.UInt8,
    'b6': pl.UInt8,
    'shared_values_count': pl.UInt8,
    'reason': pl.UInt8
  },
  orient='row'
)

TOTAL_LINES = len(dfRows)
print(f'{TOTAL_LINES=}')
display(dfRows.head(10))


TOTAL_LINES=5258


fecha,sorteo,bolillas regulares,bolillas chauchamba,cc1,cc2,id,combo,b1,b2,b3,b4,b5,b6,is_rare_combination,reason,score_left_b1,score_left_b2,score_left_b3,score_left_b4,score_left_b5,score_left_b6,score_r8_b1,score_r8_b2,score_r8_b3,score_r8_b4,score_r8_b5,score_r8_b6,shared_values,shared_values_count,score_row,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,bool,u8,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,list[f64],u8,f64,str,i64
"""22/03/2025""","""1785""","""25 07 34 29 24 23""","""36 12 20 40 06 17""","""""","""""","""072324252934""","[7, 23, … 34]",7,23,24,25,29,34,true,24,0.0455,0.0345,0.0714,0.0714,0.0714,0.0909,0.0455,0.0455,0.0455,0.04,0.0345,0.0909,"[0.0455, 0.0345]",2,0.687,"""16_01_01_04_05""",7
"""22/03/2025""","""1785""","""25 07 34 29 24 23""","""36 12 20 40 06 17""","""""","""""","""061217203640""","[6, 12, … 40]",6,12,17,20,36,40,false,0,0.0909,0.0333,0.04,0.0476,0.0526,0.25,0.0909,0.0667,0.0588,0.0312,0.0286,0.25,[],0,1.0406,"""06_05_03_16_04""",-56
"""20/03/2025""","""1784""","""13 07 35 15 03 11""","""28 12 22 35 08 37""","""""","""""","""030711131535""","[3, 7, … 35]",3,7,11,13,15,35,true,27,0.1667,0.0303,0.0333,0.037,0.0385,0.04,0.1667,0.1111,0.1,0.0909,0.0333,0.04,[0.0333],1,0.8878,"""04_04_02_02_20""",12
"""20/03/2025""","""1784""","""13 07 35 15 03 11""","""28 12 22 35 08 37""","""""","""""","""081222283537""","[8, 12, … 37]",8,12,22,28,35,37,false,0,0.0909,0.0357,0.04,0.0625,0.0909,0.2,0.0909,0.05,0.04,0.0323,0.0312,0.2,[0.04],1,0.9644,"""04_10_06_07_02""",-26
"""18/03/2025""","""1783""","""39 16 22 21 30 12""","""33 39 24 05 20 40""","""""","""""","""121621223039""","[12, 16, … 39]",12,16,21,22,30,39,false,0,0.0667,0.0417,0.0476,0.0588,0.0588,0.1,0.0667,0.0526,0.0526,0.0385,0.0294,0.1,[],0,0.7134,"""04_05_01_08_09""",-33
"""18/03/2025""","""1783""","""39 16 22 21 30 12""","""33 39 24 05 20 40""","""""","""""","""052024333940""","[5, 20, … 40]",5,20,24,33,39,40,true,16,0.0526,0.0323,0.0588,0.0714,0.1667,1.0,0.0526,0.0455,0.0333,0.0286,0.0286,1.0,[],0,2.5704,"""15_04_09_06_01""",30
"""15/03/2025""","""1782""","""19 28 03 39 10 09""","""06 35 12 28 09 18""","""""","""""","""030910192839""","[3, 9, … 39]",3,9,10,19,28,39,true,8,0.125,0.0303,0.0357,0.0357,0.05,0.0833,0.125,0.125,0.0625,0.0417,0.0294,0.0833,[0.125],1,0.8269,"""06_01_09_09_11""",31
"""15/03/2025""","""1782""","""19 28 03 39 10 09""","""06 35 12 28 09 18""","""""","""""","""060912182835""","[6, 9, … 35]",6,9,12,18,28,35,true,8,0.125,0.0333,0.0357,0.0385,0.0476,0.0833,0.125,0.1,0.0667,0.0417,0.0333,0.0833,[0.0333],1,0.8134,"""03_03_06_10_07""",-6
"""13/03/2025""","""1781""","""21 19 06 07 28 40""","""16 15 28 27 25 05""","""""","""""","""060719212840""","[6, 7, … 40]",6,7,19,21,28,40,true,16,0.1667,0.0333,0.0333,0.0526,0.0556,0.0833,0.1667,0.0588,0.0556,0.0417,0.0286,0.0833,[0.0556],1,0.8595,"""01_12_02_07_12""",-51


# ANALYZE RANDOM!
You can't analyze random bc it's random! But you can find "order" in "entropy" I guess. Like when you have your room messed up but somehow you can find everything. Nevermid.
Just look at the results below

In [7]:
unijumps = dfRows.group_by(
  'unijump'
).agg(
  pl.count('unijump').alias('count')
).sort(
  "count", descending=True
)


In [8]:
unijumps.write_excel("unijump_kbl.xlsx")

In [10]:
import json
percent_one = math.ceil(TOTAL_LINES * 0.01)
print(f'{percent_one=}')
print(unijumps[0:percent_one]['unijump'].to_list())
lst_unijumps = unijumps[0:percent_one]['unijump'].to_list()

with open('unijump_kbl.json', 'w') as f:
  f.write(json.dumps(lst_unijumps))
  

percent_one=53
[2, 5, -6, 13, -17, 3, 10, -4, -9, 21, 17, 11, 4, 16, 14, -13, -7, -10, -1, -15, -3, 0, -11, -22, -24, -14, -28, 9, -2, -12, 1, -5, 22, -18, 6, 12, 7, -19, -27, -8, -25, 8, -39, -16, -20, -23, 19, 28, -30, 15, -31, 20, -26]


In [11]:
dfRows.group_by(
  'reason'
).agg(
  pl.count('reason').alias('count')
).sort(
  "count", descending=True
)


reason,count
u8,u32
0,2716
8,1013
16,817
24,346
1,154
…,…
10,6
20,5
27,4


In [12]:
dfRows.group_by(
  'shared_values_count'
).agg(pl.count('shared_values_count').alias('count'))

shared_values_count,count
u8,u32
3,79
0,2227
4,9
1,2168
2,775


In [13]:
df_jumps_map = dfRows.group_by(
  'jumps_map'
).agg(pl.count('jumps_map').alias('count')).sort("count", descending=True)

display(df_jumps_map[0:10])

repeated_jumps_map = df_jumps_map.filter(pl.col("count") > 1)["jumps_map"].to_list()
print(f'MORE THAN 1 {len(repeated_jumps_map)=}')


q = dfRows.filter(
  pl.col("jumps_map").is_in(repeated_jumps_map) 
).sort("jumps_map")

display(q)

with open ("./jumps_map_kbl.json", "w") as file:
  json.dump(df_jumps_map["jumps_map"].to_list(), file)



jumps_map,count
str,u32
"""12_07_07_01_04""",2
"""02_01_19_01_06""",2
"""03_03_06_03_04""",2
"""08_01_03_18_04""",2
"""03_01_09_06_03""",2
"""04_06_02_11_03""",2
"""09_02_10_10_02""",2
"""02_01_09_01_03""",2
"""08_02_02_05_07""",2


MORE THAN 1 len(repeated_jumps_map)=48


fecha,sorteo,bolillas regulares,bolillas chauchamba,cc1,cc2,id,combo,b1,b2,b3,b4,b5,b6,is_rare_combination,reason,score_left_b1,score_left_b2,score_left_b3,score_left_b4,score_left_b5,score_left_b6,score_r8_b1,score_r8_b2,score_r8_b3,score_r8_b4,score_r8_b5,score_r8_b6,shared_values,shared_values_count,score_row,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,bool,u8,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,list[f64],u8,f64,str,i64
"""16/12/2023""","""1587""","""17 37 24 13 28 23""","""16 15 18 31 27 34""","""""","""""","""151618273134""","[15, 16, … 34]",15,16,18,27,31,34,false,0,0.0667,0.0476,0.0476,0.05,0.0833,0.1111,0.0667,0.0625,0.0417,0.037,0.0345,0.1111,[],0,0.7598,"""01_02_09_04_03""",34
"""05/09/2017""","""0637""","""34 18 14 19 32 33""","""13 32 29 25 14 16""","""""","""""","""131416252932""","[13, 14, … 32]",13,14,16,25,29,32,false,0,0.0769,0.0435,0.0435,0.0455,0.0714,0.0909,0.0769,0.0714,0.0455,0.04,0.037,0.0909,"[0.0455, 0.0714]",2,0.7334,"""01_02_09_04_03""",34
"""16/12/2017""","""0681""","""22 39 17 11 07 04""","""09 10 31 22 23 13""","""""","""""","""091013222331""","[9, 10, … 31]",9,10,13,22,23,31,false,0,0.1111,0.037,0.037,0.04,0.0588,0.0588,0.1111,0.0909,0.0526,0.0526,0.0385,0.0588,[0.0588],1,0.7472,"""01_03_09_01_08""",47
"""15/11/2011""","""1771""","""04 05 08 17 18 26""","""""","""""","""""","""040508171826""","[4, 5, … 26]",4,5,8,17,18,26,true,16,0.25,0.0312,0.0312,0.0333,0.0455,0.0455,0.25,0.1667,0.0714,0.0714,0.0476,0.0455,[0.0455],1,1.0893,"""01_03_09_01_08""",47
"""31/10/2019""","""0974""","""34 18 21 36 27 39""","""18 28 25 13 14 35""","""""","""""","""131418252835""","[13, 14, … 35]",13,14,18,25,28,35,false,0,0.0769,0.0435,0.0435,0.05,0.0714,0.0833,0.0769,0.0625,0.0455,0.0417,0.0333,0.0833,[],0,0.7118,"""01_04_07_03_07""",22
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""20/04/2007""","""821""","""05 19 34 26 37 21""","""""","""""","""""","""051921263437""","[5, 19, … 37]",5,19,21,26,34,37,false,0,0.0556,0.0323,0.0556,0.0588,0.0769,0.1667,0.0556,0.0526,0.0435,0.0333,0.0312,0.1667,[],0,0.8288,"""14_02_05_08_03""",7
"""15/06/2017""","""0602""","""16 26 18 20 29 01""","""07 13 33 20 15 06""","""""","""""","""011618202629""","[1, 16, … 29]",1,16,18,20,26,29,true,24,0.0667,0.0286,0.0476,0.05,0.0526,0.0714,0.0667,0.0625,0.0588,0.0455,0.0417,0.0714,[],0,0.6635,"""15_02_02_06_03""",-2
"""13/03/2012""","""1805""","""03 18 20 22 28 31""","""""","""""","""""","""031820222831""","[3, 18, … 31]",3,18,20,22,28,31,true,8,0.0588,0.0303,0.0526,0.0556,0.0588,0.0833,0.0588,0.0556,0.0526,0.0417,0.0385,0.0833,"[0.0526, 0.0556]",2,0.6699,"""15_02_02_06_03""",-2


In [14]:
def show_b_stats (col: str):
  df_viz = dfRows.group_by(
    col
  ).agg(pl.count(col).alias('count'))

  chart = df_viz.plot.bar(
    x=col,
    y='count',
    color=col,
  )

  display(df_viz)
  display(chart)



In [15]:
show_b_stats('score_left_b1')
show_b_stats('score_r8_b1')


score_left_b1,count
f64,u32
0.0476,93
0.0333,3
0.125,383
0.0588,148
0.0769,266
…,…
0.0909,325
0.1667,352
0.0417,41


alt.Chart(...)

score_r8_b1,count
f64,u32
0.0769,266
0.0333,3
0.0476,93
0.125,383
0.0588,148
…,…
0.1667,352
0.0909,325
0.0526,124


alt.Chart(...)

In [16]:
show_b_stats('score_left_b2')
show_b_stats('score_r8_b2')

score_left_b2,count
f64,u32
0.0588,31
0.0769,7
0.125,1
0.0333,353
0.0476,67
…,…
0.1667,1
0.0909,2
0.0417,132


alt.Chart(...)

score_r8_b2,count
f64,u32
0.0588,270
0.0769,292
0.0333,34
0.0476,199
0.125,198
…,…
0.0385,110
0.1667,138
0.0417,167


alt.Chart(...)

In [17]:
show_b_stats('score_left_b3')
show_b_stats('score_r8_b3')

score_left_b3,count
f64,u32
0.0476,221
0.0588,124
0.0333,352
0.125,15
0.0769,59
…,…
0.1667,3
0.0385,301
0.0556,156


alt.Chart(...)

score_r8_b3,count
f64,u32
0.0333,130
0.0476,288
0.0588,249
0.125,68
0.0769,177
…,…
0.0385,256
0.1667,46
0.0526,289


alt.Chart(...)

In [18]:
show_b_stats('score_left_b4')
show_b_stats('score_r8_b4')

score_left_b4,count
f64,u32
0.0333,138
0.0769,176
0.0476,285
0.125,57
0.0588,248
…,…
0.1667,34
0.0909,121
0.0417,249


alt.Chart(...)

score_r8_b4,count
f64,u32
0.0769,56
0.0588,113
0.0333,321
0.0476,230
0.125,9
…,…
0.0909,39
0.0385,340
0.0556,158


alt.Chart(...)

In [19]:
show_b_stats('score_left_b5')
show_b_stats('score_r8_b5')

score_left_b5,count
f64,u32
0.0476,230
0.0333,46
0.0769,276
0.0588,289
0.125,209
…,…
0.0385,86
0.1667,130
0.0556,269


alt.Chart(...)

score_r8_b5,count
f64,u32
0.0588,27
0.0769,10
0.0476,84
0.125,1
0.0333,358
…,…
0.0385,176
0.1667,1
0.0556,28


alt.Chart(...)